# Load, select, clean json data 

In [2]:
import json 
import re
import os

In [3]:
file_name = "lens-caps.json"
base_name = os.path.splitext(os.path.basename(file_name))[0]


In [4]:
with open(file_name,'r',encoding="utf-8") as f:
    data = json.load(f)

In [5]:
item_list = data["mods"]["listItems"]

In [72]:
item_list[0]

{'name': 'The lens is suitable for all cameras with sizes 7, 72, 75, 49, 5 quality, 40.ng, and includes 2025新品 one high Wenefa2 55, 58, 62, 67, 82mm special price center clamp quick-release cap.',
 'nid': '489419476',
 'itemId': '489419476',
 'icons': [],
 'image': 'https://static-01.daraz.com.np/p/f6485b5ba6430f883f5d828256e2f9ff.png',
 'isSmartImage': False,
 'originalPriceShow': '',
 'priceShow': 'Rs. 1,520',
 'ratingScore': '',
 'review': '',
 'location': 'Overseas',
 'thumbs': [],
 'sellerName': 'Manhuayuangu',
 'sellerId': '900270048480',
 'brandName': 'No Brand',
 'brandId': '39704',
 'cheapest_sku': '489419476_NP-2198959038',
 'skuId': '2198959038',
 'sku': '489419476_NP',
 'categories': [240, 8217, 8220, 8223],
 'price': '1520',
 'inStock': True,
 'originalPrice': '1824',
 'clickTrace': 'query:camera;nid:489419476;src:LazadaMainSrp;rn:3e3ad865f4f6894a6f06a36a8a95a2fe;region:np;sku:489419476_NP;price:1520;client:desktop;supplier_id:900270048480;biz_source:h5_unknow;slot:0;utlog

In [6]:
type(item_list)

list

In [6]:
def chunking_each_doc(doc,chunk_size = 128):
    start = 0
    doc = json.dumps(doc)
    tokens = re.findall(r'\w+|[{}[\]:,",]',doc)
    chunks = []
    while start <= len(tokens):
        end = min(start+chunk_size,len(tokens))
        chunk = " ".join(tokens[start:end])
        chunks.append(chunk)
        start+=chunk_size
    return chunks

In [7]:
all_chunks = []
for item in item_list:
    chunks_of_each_doc= chunking_each_doc(item)
    all_chunks.append(chunks_of_each_doc)


In [8]:
all_chunks

[['{ " name " : " The lens is suitable for all cameras with sizes 7 , 72 , 75 , 49 , 5 quality , 40 ng , and includes 2025 u65b0 u54c1 one high Wenefa2 55 , 58 , 62 , 67 , 82mm special price center clamp quick release cap " , " nid " : " 489419476 " , " itemId " : " 489419476 " , " icons " : [ ] , " image " : " https : static 01 daraz com np p f6485b5ba6430f883f5d828256e2f9ff png " , " isSmartImage " : false , " originalPriceShow " : " " , " priceShow " : " Rs 1 , 520 " , " ratingScore " : " " , " review',
  '" : " " , " location " : " Overseas " , " thumbs " : [ ] , " sellerName " : " Manhuayuangu " , " sellerId " : " 900270048480 " , " brandName " : " No Brand " , " brandId " : " 39704 " , " cheapest_sku " : " 489419476_NP 2198959038 " , " skuId " : " 2198959038 " , " sku " : " 489419476_NP " , " categories " : [ 240 , 8217 , 8220 , 8223 ] , " price " : " 1520 " , " inStock " : true , " originalPrice " : " 1824 " , " clickTrace " : " query : camera nid : 489419476 src : LazadaMainSrp

In [75]:
len(all_chunks)

40

In [76]:
all_chunks[0][0]

'{ " name " : " The lens is suitable for all cameras with sizes 7 , 72 , 75 , 49 , 5 quality , 40 ng , and includes 2025 u65b0 u54c1 one high Wenefa2 55 , 58 , 62 , 67 , 82mm special price center clamp quick release cap " , " nid " : " 489419476 " , " itemId " : " 489419476 " , " icons " : [ ] , " image " : " https : static 01 daraz com np p f6485b5ba6430f883f5d828256e2f9ff png " , " isSmartImage " : false , " originalPriceShow " : " " , " priceShow " : " Rs 1 , 520 " , " ratingScore " : " " , " review'

In [77]:
all_chunks[0]

['{ " name " : " The lens is suitable for all cameras with sizes 7 , 72 , 75 , 49 , 5 quality , 40 ng , and includes 2025 u65b0 u54c1 one high Wenefa2 55 , 58 , 62 , 67 , 82mm special price center clamp quick release cap " , " nid " : " 489419476 " , " itemId " : " 489419476 " , " icons " : [ ] , " image " : " https : static 01 daraz com np p f6485b5ba6430f883f5d828256e2f9ff png " , " isSmartImage " : false , " originalPriceShow " : " " , " priceShow " : " Rs 1 , 520 " , " ratingScore " : " " , " review',
 '" : " " , " location " : " Overseas " , " thumbs " : [ ] , " sellerName " : " Manhuayuangu " , " sellerId " : " 900270048480 " , " brandName " : " No Brand " , " brandId " : " 39704 " , " cheapest_sku " : " 489419476_NP 2198959038 " , " skuId " : " 2198959038 " , " sku " : " 489419476_NP " , " categories " : [ 240 , 8217 , 8220 , 8223 ] , " price " : " 1520 " , " inStock " : true , " originalPrice " : " 1824 " , " clickTrace " : " query : camera nid : 489419476 src : LazadaMainSrp',

In [12]:
type(all_chunks)

list

# Qdrant Operation for Hybrid Search

In [ ]:
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer

In [15]:
dense_model_name = "sentence-transformers/all-MiniLM-L6-v2"
sparse_model_name = "prithivida/Splade_PP_en_v1"

In [16]:
client = QdrantClient(url = "http://localhost:6333")

In [56]:
if not client.collection_exists("hybrid_search_daraz"):
    client.create_collection(
        collection_name = "hybrid_search_daraz",
        vectors_config = {
            "dense" : models.VectorParams(size = client.get_embedding_size(dense_model_name), distance = models.Distance.COSINE)

        },
        sparse_vectors_config= {
            "sparse" : models.SparseVectorParams()
        }
    )

In [40]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [78]:
# creation of embeddings

#dense vectors and sparse vectors of the same document and chunks
final_dense_vector = []
final_sparse_vector = []
for i in range(len(all_chunks)):
        dense_embeddings = encoder.encode(all_chunks[i],show_progress_bar=True,convert_to_numpy=True)
        final_dense_vector.append(dense_embeddings)





Batches: 100%|██████████| 1/1 [00:00<00:00,  7.73it/s]


In [79]:
final_dense_vector

[array([[-0.01725098,  0.01887227, -0.04758353, ..., -0.01119488,
         -0.11504184,  0.06792808],
        [-0.01740227, -0.01158251, -0.08259612, ..., -0.0480869 ,
         -0.08224002,  0.10919204],
        [-0.02663496,  0.07192978, -0.07561843, ..., -0.0192584 ,
         -0.0247532 ,  0.04867316],
        [-0.06852859,  0.04353247, -0.1091061 , ..., -0.03919964,
         -0.02819419,  0.04249191],
        [ 0.03060741, -0.03198544, -0.02841446, ..., -0.09633103,
         -0.05300676,  0.06436794]], shape=(5, 384), dtype=float32),
 array([[ 0.01110107,  0.03198135, -0.02528231, ..., -0.04988011,
         -0.09079593,  0.03829745],
        [-0.06231314,  0.01525998, -0.09074757, ..., -0.02340672,
         -0.06704278,  0.07382487],
        [-0.03285393,  0.06424579, -0.06658939, ..., -0.02639928,
         -0.02288486,  0.06052505],
        [-0.05790181,  0.02922601, -0.08560897, ..., -0.01738248,
          0.02005384,  0.04824462],
        [-0.00626088,  0.01350834, -0.01631294, .

In [57]:
client.create_payload_index(
    collection_name = "hybrid_search_daraz",
    field_name = "doc_id",
    field_schema = "integer"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [58]:
client.create_payload_index(
    collection_name = "hybrid_search_daraz",
    field_name = "chunk_id",
    
    field_schema = "integer"
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [59]:
client.create_payload_index(
    collection_name = "hybrid_search_daraz",
    field_name = "file_name",
    field_schema = "keyword"
)

UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [68]:
from fastembed.embedding import SparseEmbedding

ImportError: cannot import name 'SparseEmbedding' from 'fastembed.embedding' (c:\Users\royka\.conda\envs\rag\lib\site-packages\fastembed\embedding.py)

In [80]:
offset = 0
info = client.get_collection(collection_name = "hybrid_search_daraz")
file_number = 0
count = info.points_count
if(count!=0):
    res, _ = client.scroll(
        collection_name = "hybrid_search_daraz",
        with_payload = True,
        with_vectors=False,
        limit = 1,
        order_by = {
            "key":"chunk_id",
            "direction" : "desc"
        }
)
    
    if(res):
        file_number = res[0].payload.get("doc_id")[0]
        file_number = file_number + 1
        last_id = res[0].id
        offset = last_id + 1
    else:
        offset = 0
else:
    offset = 0
    file_number = 0


for doc in range(len(all_chunks)):
    for idx in range(len(all_chunks[doc])):
        client.upsert(
            collection_name = "hybrid_search_daraz",
            points = [
                models.PointStruct(
                    id = offset,
                    payload = {
                        "doc_id" : (file_number,doc),
                        "chunk_id":offset,
                        "chunk" : all_chunks[doc][idx],
                        "file_name" : base_name
                    },
                    vector = {
                        "dense" : final_dense_vector[doc][idx].tolist(),
                        "sparse" : models.Document(text = all_chunks[doc][idx],model = sparse_model_name)
                    }
                )
            ]
        )
        offset+=1

In [1]:
from fastembed.embedding import SparseEmbedding

c:\Users\royka\.conda\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-06 16:36:18.625 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


ImportError: cannot import name 'SparseEmbedding' from 'fastembed.embedding' (c:\Users\royka\.conda\envs\rag\lib\site-packages\fastembed\embedding.py)

In [2]:
from fastembed import TextEmbedding


In [7]:
sparse_model_name = "Qdrant/bge-sparse-en-v1.5"
sparse_model = TextEmbedding(model_name=sparse_model_name)

ValueError: Model Qdrant/bge-sparse-en-v1.5 is not supported in TextEmbedding. Please check the supported models using `TextEmbedding.list_supported_models()`

In [9]:
from fastembed import TextEmbedding

print("--- Verifying fastembed installation ---")

supported_models = TextEmbedding.list_supported_models()

found_sparse = False
print("\nChecking for supported sparse models...")
for model_info in supported_models:
    # We are looking for the official sparse models
    if "sparse" in model_info["model"] or "bm25" in model_info["model"]:
        print(f"  [SUCCESS] Found sparse model: {model_info['model']}")
        found_sparse = True

if not found_sparse:
    print("\n  [FAILURE] No sparse models found. The installation may not have updated correctly.")
else:
    print("\n[SUCCESS] Your fastembed installation now supports both dense and sparse models!")

--- Verifying fastembed installation ---

Checking for supported sparse models...

  [FAILURE] No sparse models found. The installation may not have updated correctly.


In [5]:
from fastembed import TextEmbedding


supported_models = TextEmbedding.list_supported_models()


for model_info in supported_models:
    print(model_info)

{'model': 'BAAI/bge-base-en', 'sources': {'hf': 'Qdrant/fast-bge-base-en', 'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz', '_deprecated_tar_struct': True}, 'model_file': 'model_optimized.onnx', 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.', 'license': 'mit', 'size_in_GB': 0.42, 'additional_files': [], 'dim': 768, 'tasks': {}}
{'model': 'BAAI/bge-base-en-v1.5', 'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q', 'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz', '_deprecated_tar_struct': True}, 'model_file': 'model_optimized.onnx', 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.', 'license': 'mit', 'size_in_GB': 0.21, 'additional_files': [], 'dim': 768, 'tasks': {}}
{'model': 'BAAI/bge-large-en-v1.5', 'sources': {'hf': 'qdr

In [47]:
len(all_chunks)

40

# Searching Logic

In [87]:
class HybridSearch:
    
    def __init__(self,collection_name):
        self.collection_name = collection_name
        self.client = QdrantClient(url = "http://localhost:6333")
        self.encoder_dense = SentenceTransformer("all-MiniLM-L6-v2")
        self.sparse_model = "prithivida/Splade_PP_en_v1"
    def search_points(self,item:str):
        search_result = self.client.query_points(
            collection_name = self.collection_name,
            query = models.FusionQuery(
                fusion = models.Fusion.RRF
            ),
            prefetch=[
                models.Prefetch(
                    query= self.encoder_dense.encode([item])[0].tolist(),
                    using = "dense",
                    
                ),
                models.Prefetch(
                    query = models.Document(text = item,model = self.sparse_model),
                    using = "sparse",
                )
                ],
                query_filter=None,
                limit = 3
        ).points

        dense_result = self.client.query_points(
        collection_name=self.collection_name,
        query=self.encoder_dense.encode([item])[0].tolist(),
        using="dense",
        limit=3
        ).points

        sparse_result = self.client.query_points(
        collection_name=self.collection_name,
        query=models.Document(text=item, model=self.sparse_model),
        using="sparse",
        limit=3
        ).points

        for point in search_result:
            print("score = ",point.score, "Point_id=", point.id, "doc_id = ", point.payload.get("doc_id"), "file_name = ",point.payload.get("file_name"), "chunk_id = " , point.payload.get("chunk_id"))
        for point in dense_result:
            print("score = ",point.score, "Point_id=", point.id, "doc_id = ", point.payload.get("doc_id"), "file_name = ",point.payload.get("file_name"), "chunk_id = " , point.payload.get("chunk_id"))
        for point in sparse_result:
            print("score = ",point.score, "Point_id=", point.id, "doc_id = ", point.payload.get("doc_id"), "file_name = ",point.payload.get("file_name"), "chunk_id = " , point.payload.get("chunk_id"))
        


In [83]:
client.create_payload_index(
    collection_name= "hybrid_search_daraz",
    field_name = "doc_id",
    field_schema = "integer"
)

UpdateResult(operation_id=471, status=<UpdateStatus.COMPLETED: 'completed'>)

In [89]:
search_item = HybridSearch("hybrid_search_daraz")
search_item.search_points("lens-cap")

score =  0.5 Point_id= 267 doc_id =  [1, 1] file_name =  lens-caps chunk_id =  267
score =  0.5 Point_id= 434 doc_id =  [1, 34] file_name =  lens-caps chunk_id =  434
score =  0.33333334 Point_id= 268 doc_id =  [1, 1] file_name =  lens-caps chunk_id =  268
score =  0.5274915 Point_id= 267 doc_id =  [1, 1] file_name =  lens-caps chunk_id =  267
score =  0.50564957 Point_id= 277 doc_id =  [1, 3] file_name =  lens-caps chunk_id =  277
score =  0.50542873 Point_id= 302 doc_id =  [1, 8] file_name =  lens-caps chunk_id =  302
score =  19.040468 Point_id= 434 doc_id =  [1, 34] file_name =  lens-caps chunk_id =  434
score =  19.022373 Point_id= 268 doc_id =  [1, 1] file_name =  lens-caps chunk_id =  268
score =  18.919115 Point_id= 308 doc_id =  [1, 9] file_name =  lens-caps chunk_id =  308
